In [0]:
from google.colab import files, auth, drive

In [0]:
# Mount to drive
drive.mount('/content/gdrive', force_remount=True)
data_dir_drive ='/content/gdrive/My Drive/Colab Notebooks/NLP1/Lab5'

In [1]:
! pip install nltk

In [6]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> wordnet
      Unzipping corpora/wordnet.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

# **Lesk**
Lesk measure is used to measure the relatedness of two words(senses) by counting the number of words they have in common (overlaps), in their definitions (glosses). The Lesk measure is the number of such common words.

Lesk algorithm is used in word disambiguation; it associates a sense to a given word based on how related it is to the context (the rest of the words in the text).

Lesk measure is already implemented in nltk:

In [7]:
from nltk.wsd import lesk

sys = lesk(nltk.word_tokenize('Students enjoy going to school, studying and reading books'),'school')
print(sys)
print(sys.definition())

Synset('school.v.01')
educate in or as if in a school


**Extended Gloss Overlaps (Extended Lesk)**
This technique was presented by Satanjeev Banerjee and Ted Pedersen in 2003 in an [article](https://www.researchgate.net/publication/2563220_Extended_Gloss_Overlaps_as_a_Measure_of_Semantic_Relatedness).

The algorithm measures the relatedness of two words. Just like Lesk, it counts the overlaps of glosses, however it takes into account the related glosses of the two word as well.

Suppose that we have two synsets s1 and s2. For both of them we obtain the glosses of the synsets for all:

hypernyms
hyponyms
meronyms
holonyms
troponyms
attributes
similar–to
also–see
In computing the score, for each single word that appears in both glosses we add 1. However if it appears in a common phrase, supposing the length of common phrase is L, we add L2(for example, if "white bread" appears in both glosses, we add 4). We obviusly don't add the score for the separate words in the phrase. We try to find the longest common phrase (tht doesn't start or end with a pronoun, preposition, article or conjunction in both glosses.

If we have multiple synsets in the same relation to one of the given synsets we create a string with all the glosses concatenated.

# **Exercises and homework**

# **1**
Implement Original Lesk algorithm with the help of a function that computes the score for two given glosses. For a given text and a given word, try to find the sense of that word, considering the Lesk measure. Check your result with the already implemented (simplified) lesk algorithm in nltk. You may have different results, as the simplified Lesk algorithm compares the target word glosses with the words from the context (not their definitions).

In [8]:
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
from nltk.corpus import wordnet

def compare_glosses(gloss1, gloss2):
    words1 = word_tokenize(gloss1)
    words2 = word_tokenize(gloss2)

    words1 = [x.lower() for x in words1 if x.isalnum()]
    words2 = [x.lower() for x in words2 if x.isalnum()]

    #print(gloss1, gloss2)
    #print(words1, words2)

    ps=nltk.PorterStemmer()

    stem1 = list(map(lambda x: ps.stem(x), words1))
    stem2 = list(map(lambda x: ps.stem(x), words2))

    #print(stem1, stem2)

    r = 0;
    for s1 in stem1:
        for s2 in stem2:
            if s1 == s2:
                r += 1

    return r

c1 = compare_glosses(wordnet.synsets('school')[0].definition(), wordnet.synsets('school')[1].definition())
c2 = compare_glosses(wordnet.synsets('school')[1].definition(), wordnet.synsets('school')[3].definition())
c3 = compare_glosses(wordnet.synsets('school')[0].definition(), wordnet.synsets('cat')[0].definition())

print(c1, c2, c3)

1 2 0


In [9]:
def myLesk(context, word):
    synsets = wordnet.synsets(word)

    maximum = 0
    syn = None

    for s in synsets:
        m = 0
        for word_context in context:
            for word_sense in wordnet.synsets(word_context):
                m += compare_glosses(s.definition(), word_sense.definition())
        print(m, s.definition())
        if m > maximum:
            maximum = m
            syn = s
    return syn

sys = myLesk(nltk.word_tokenize('Students enjoy going to school, studying and reading books'),'school')
print(sys)
print(sys.definition())

16 an educational institution
76 a building where young people receive education
161 the process of being formally educated at a school
348 a body of creative artists or writers or thinkers linked by a similar style or by similar teachers
227 the period of instruction in a school; the time period when school is in session
31 an educational institution's faculty and students
109 a large group of fish
190 educate in or as if in a school
169 teach or refine to be discriminative in taste or judgment
193 swim in or form a large group of fish
Synset('school.n.04')
a body of creative artists or writers or thinkers linked by a similar style or by similar teachers


In [10]:
sys = myLesk(nltk.word_tokenize('The reactor explosion killed two of the reactor operating staff.'),'nuclear')
print(sys)
print(sys.definition())

49 (weapons) deriving destructive energy from the release of atomic energy
110 of or relating to or constituting the nucleus of an atom
126 of or relating to or constituting the nucleus of a cell
38 constituting or like a nucleus
Synset('nuclear.a.03')
of or relating to or constituting the nucleus of a cell


# **2**
Implement extended Lesk algorithm. For a list of 7-10 words, print the measure for each pair of words (without repeating the words). Just like in the former exercise, try to obtain the word sense for the given text and word. Can you find a text and word where simple Lesk gives the wrong answer, however extended Lesk gives the right answer?

In [11]:
def compare_glosses_overlap(gloss1, gloss2):
    words1 = word_tokenize(gloss1)
    words2 = word_tokenize(gloss2)

    words1 = [x.lower() for x in words1 if x.isalnum()]
    words2 = [x.lower() for x in words2 if x.isalnum()]

    #print(gloss1, gloss2)
    #print(words1, words2)

    ps=nltk.PorterStemmer()

    stem1 = list(map(lambda x: ps.stem(x), words1))
    stem2 = list(map(lambda x: ps.stem(x), words2))

    if len(stem1) > len(stem2):
        stem1, stem2 = stem2, stem1

    #print(stem1, stem2)

    r = 0;
    for size in range(len(stem1))[::-1]: #revese order
        for i in range(len(stem1)):
            if i + size == len(stem1):
                break
            patch1 = stem1[i:i + size + 1]
            for j in range(len(stem2) - size):
                patch2 = stem2[j:j + size + 1]
                if patch1 == patch2:
                    r += len(patch1) ** 2
                    stem1 = stem1[:i] + stem1[i + size + 1:] #cut what I found
                    stem2 = stem2[:j] + stem2[j + size + 1:]

    return r

c1 = compare_glosses_overlap('w1 w2 w3 w4 w5 w6 w7 w8', 'w10 w1 w2 w11 w5 w6 w7 w12 w8')
c2 = compare_glosses_overlap(wordnet.synsets('school')[0].definition(), wordnet.synsets('school')[1].definition())
c3 = compare_glosses_overlap(wordnet.synsets('school')[1].definition(), wordnet.synsets('school')[3].definition())
c4 = compare_glosses_overlap(wordnet.synsets('school')[0].definition(), wordnet.synsets('cat')[0].definition())

print(c1, c2, c3, c4)

14 1 2 0


In [17]:
def compute_extended_sense(syn):
    hypernyms = syn.hypernyms()
    hyponyms = syn.hyponyms()
    holonyms = syn.substance_holonyms() + syn.part_holonyms() + syn.part_holonyms()
    meronyms = syn.substance_meronyms() + syn.part_meronyms() + syn.part_meronyms()
    troponyms = syn.entailments()
    attributes = syn.attributes()
    similar_to = syn.similar_tos()
    also_see = syn.also_sees()

    hypernym_s = '' #concatenate deffinitions
    for hypernym in hypernyms:
        hypernym_s += hypernym.definition()
    hyponym_s = ''
    for hyponym in hyponyms:
        hyponym_s += hyponym.definition()
    holonym_s = ''
    for holonym in holonyms:
        holonym_s += holonym.definition()
    meronym_s = ''
    for meronym in meronyms:
        meronym_s += meronym.definition()
    troponym_s = ''
    for troponym in troponyms:
        troponym_s += troponym.definition()
    attribute_s = ''
    for attribute in attributes:
        attribute_s += attribute.definition()
    similar_to_s = ''
    for e in similar_to:
        similar_to_s += e.definition()
    also_see_s = ''
    for e in also_see:
        also_see_s += e.definition()

    return [syn.definition(),
            hypernym_s,
            hyponym_s,
            holonym_s,
            meronym_s,
            troponym_s,
            attribute_s,
            similar_to_s,
            also_see_s]

compute_extended_sense(wordnet.synset('water.n.01'))

['binary compound that occurs at room temperature as a clear colorless odorless tasteless liquid; freezes into ice below 0 degrees centigrade and boils above 100 degrees centigrade; widely used as a solvent',
 'chemical compound composed of only two elementsfluid matter having no fixed shape but a fixed volume',
 'water used for a bathwater accumulated in the bilge of a shipwater in which dishes and cooking utensils are washedwater that has been purified by distillationwater that is not saltyunderground water that is held in the soil and in pervious rockswater that contains mineral salts (as calcium and magnesium ions) that limit the formation of lather with soapwater that has been blessed by a priest for use in symbolic purificationsolution of calcium hydroxide in water used as an antacidmelted snow or icewater containing saltspartially melted snowwater that is not hard (does not contain mineral salts that interfere with the formation of lather with soap)water directly from the spigot

In [18]:
def myExtendedLesk(context, word):
    synsets = wordnet.synsets(word)

    maximum = 0
    syn = None

    for s in synsets:
        m = 0
        L = compute_extended_sense(s)
        for word_context in context:
            for word_sense in wordnet.synsets(word_context):
                L_word_sense = compute_extended_sense(word_sense)
                for l in L: # cartesian product
                    for l_w in L_word_sense:
                        m += compare_glosses_overlap(l, l_w)
        print(m, s.definition())
        if m > maximum:
            maximum = m
            syn = s
    return syn

sys = myExtendedLesk(nltk.word_tokenize('Students enjoy going to school, studying and reading books'),'school')
print(sys)
print(sys.definition())

66926 an educational institution
4257 a building where young people receive education
1085 the process of being formally educated at a school
26395 a body of creative artists or writers or thinkers linked by a similar style or by similar teachers
2776 the period of instruction in a school; the time period when school is in session
369 an educational institution's faculty and students
974 a large group of fish
1569 educate in or as if in a school
1254 teach or refine to be discriminative in taste or judgment
783 swim in or form a large group of fish
Synset('school.n.01')
an educational institution


In [19]:
sys = myLesk(nltk.word_tokenize('The reactor explosion killed two of the reactor operating staff.'),'nuclear')
print(sys)
print(sys.definition())

49 (weapons) deriving destructive energy from the release of atomic energy
110 of or relating to or constituting the nucleus of an atom
126 of or relating to or constituting the nucleus of a cell
38 constituting or like a nucleus
Synset('nuclear.a.03')
of or relating to or constituting the nucleus of a cell


In [20]:
sys = myExtendedLesk(nltk.word_tokenize('The reactor explosion killed two of the reactor operating staff.'),'nuclear')
print(sys)
print(sys.definition())

207 (weapons) deriving destructive energy from the release of atomic energy
272 of or relating to or constituting the nucleus of an atom
369 of or relating to or constituting the nucleus of a cell
444 constituting or like a nucleus
Synset('nuclear.s.04')
constituting or like a nucleus


In [21]:
sys = lesk(nltk.word_tokenize('The reactor explosion killed two of the reactor operating staff.'),'nuclear')
print(sys)
print(sys.definition())

Synset('nuclear.a.03')
of or relating to or constituting the nucleus of a cell
